In [1]:
import sys
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('expand_frame_repr', False)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import datetime as dt
import sklearn
import re

In [2]:
if os.getcwd()[9:14] == 'ssoma':
    file_path = "C:\\Users\\ssoma\\OneDrive - Bayer\\Personal Data-R90PQ30R\\Analytics\\house-pricing\\input-data\\st-charles\\"
else:
    file_path = "C:\\Users\\sivac\\Documents\\Analytics\\house-pricing\\input-data\\st-charles\\"


file_list = os.listdir(file_path)
for i in file_list:
    i = file_path + i
    file = pd.read_csv(i)
    
    if 'df' in globals():
        df = pd.concat([df, file])
    else:
        df = file

print(f'Raw dataset shape: {df.shape}')

df_new = df[['MAILING_ADDR', 'MAILING_ZIP', 'SUBDIVISION', 'SCHOOL_DISTRICT', 'LOT_SIZE', 'YEAR_BUILT', 'QUALITY_CODE', 'BEDROOMS', 'BATHROOMS',
            'HALF_BATHROOMS', 'TOTAL_ROOMS', 'ARCHITECTURAL_STYLE', 'TOTAL_AREA', 'BASE_AREA', 'BASEMENT_AREA','BASEMENT_FINISHED_AREA',
            'PARKING_AREA', 'LAND_VALUE', 'LAST_SALE_PRICE', 'LAST_SALE_DATE', 'MARKET_VALUE', 'LAST_SALE_UPSF']]

df_new = df_new.drop_duplicates().reset_index(drop=True)
print(f'Raw dataset shape with duplicates removed: {df_new.shape}')
df_new.head()

Raw dataset shape: (5423, 32)
Raw dataset shape with duplicates removed: (5171, 22)


,MAILING_ADDR,MAILING_ZIP,SUBDIVISION,SCHOOL_DISTRICT,LOT_SIZE,YEAR_BUILT,QUALITY_CODE,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,ARCHITECTURAL_STYLE,TOTAL_AREA,BASE_AREA,BASEMENT_AREA,BASEMENT_FINISHED_AREA,PARKING_AREA,LAND_VALUE,LAST_SALE_PRICE,LAST_SALE_DATE,MARKET_VALUE,LAST_SALE_UPSF
0,1701 N 3RD ST,63301-2119,ALEXANDERS SUB ST CHAS,St Charles,40X150,2005,R - 30 - Average,3,2,0,5,2STY - 2 Story Building,"1,016",508,508,0,276,"$25,000.00","$118,750.00",01/16/2018,"$141,622.00",$116.88
1,901 S 5TH ST,63301,NaN,St Charles,0.3086 AC,2005,C - Not used,0,0,0,0,MEDICAL OFFICE,"2,994","2,994",0,0,0,"$110,905.00",$0.00,01/18/2005,"$561,853.00",$0.00
2,707 WATER ST,63301,NaN,St Charles,40X117,2005,R - 35 - Average to Good,4,3,1,7,15ST - 1 1/2 Story Building,"1,722","1,722",754,380,440,"$21,429.00","$305,300.00",03/11/2021,"$249,965.00",$177.29
3,100 N KINGSHIGHWAY,63301,OREARS SUB ST CHAS,St Charles,0.4400 AC,2005,C - Not used,0,0,0,0,SNACK BAR,"2,000","2,000",0,0,0,"$158,120.00","$1,100,000.00",04/07/2021,"$398,955.00",$550.00
4,2800 ELM ST,63301,NaN,St Charles,5.0000 AC,2005,C - Not used,0,0,0,0,"SHED, EQUIPMENT",176,176,0,0,0,"$2,062.00",$0.00,NaN,"$79,884.00",$0.00


In [9]:
df.ZIP.unique()

array([63301, 63303, 63376,  6330], dtype=int64)

In [3]:
# Reading unseen dataset
unseen_data = file_path.replace("input-data\\st-charles\\","")
unseen_data = pd.read_csv(unseen_data + "unseen_data.csv")

# Filtering source dataset as per unseen data

subdivision = unseen_data['SUBDIVISION'].values
address = unseen_data['MAILING_ADDR'].values

df_new = df_new[df_new['SUBDIVISION'].isin(subdivision)].copy()
df_new = df_new[~df_new['MAILING_ADDR'].isin(address)].copy().reset_index(drop=True)


In [4]:
df_new = df_new[df_new.QUALITY_CODE != 'C - Not used'].copy().reset_index(drop=True) # Removing any non residential data
df_backup = df_new.copy()
df_new.head()

,MAILING_ADDR,MAILING_ZIP,SUBDIVISION,SCHOOL_DISTRICT,LOT_SIZE,YEAR_BUILT,QUALITY_CODE,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,ARCHITECTURAL_STYLE,TOTAL_AREA,BASE_AREA,BASEMENT_AREA,BASEMENT_FINISHED_AREA,PARKING_AREA,LAND_VALUE,LAST_SALE_PRICE,LAST_SALE_DATE,MARKET_VALUE,LAST_SALE_UPSF
0,20 TOWERBRIDGE PL,63303,TALBRIDGE VLGS A-C,Francis Howell,0.3000 AC,2006,R - 45 - Good to Very Good,4,3,2,10,2STY - 2 Story Building,"3,304","2,132","2,132",580,650,$0.00,"$465,606.00",05/01/2007,"$480,260.00",$140.92
1,11700 LAKESHORE DR,63141,TALBRIDGE VLGS A-C,Francis Howell,0.5500 AC,2006,R - 45 - Good to Very Good,4,3,1,10,2STY - 2 Story Building,"4,000","2,186","1,898",0,620,$0.00,"$515,000.00",08/16/2021,"$521,715.00",$128.75
2,65 TOWERBRIDGE PL,63303,TALBRIDGE VLGS A-C,Francis Howell,0.5500 AC,2006,R - 45 - Good to Very Good,4,3,1,10,2STY - 2 Story Building,"3,952","2,168","1,868",0,940,$0.00,"$645,000.00",08/11/2020,"$528,630.00",$163.21
3,5 TOWERBRIDGE PL,63303,TALBRIDGE VLGS A-C,Francis Howell,0.2500 AC,2006,R - 45 - Good to Very Good,4,4,1,9,2STY - 2 Story Building,"4,252","2,336","1,988","1,073",620,$0.00,"$499,900.00",10/01/2018,"$585,120.00",$117.57
4,3 TOWERBRIDGE PL,63303,TALBRIDGE VLGS A-C,Francis Howell,0.2500 AC,2006,R - 45 - Good to Very Good,3,2,0,9,1STY - 1 Story Building,"2,746","2,746","2,702",0,620,$0.00,"$375,000.00",04/05/2018,"$447,057.00",$136.56


In [5]:
df_new = df_backup.copy()

# Removing any rows without sale year

df_new = df_new[df_new.LAST_SALE_DATE.notnull()].copy().reset_index(drop=True)


# Fixing the lot size
df_new['start'] = np.where(df_new.LOT_SIZE.str.find('AC') == -1, df_new.LOT_SIZE.str.split('X') , df_new.LOT_SIZE.str.replace(pat=" AC", repl=""))
df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1] = df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1].apply(lambda x: list(map(float, x)))
df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1]  = df_new['start'][df_new.LOT_SIZE.str.find('AC') == -1].apply(lambda x: np.prod(x[0:2])/43560)
df_new['LOTS_SIZE'] = pd.to_numeric(df_new['start'])

# Fixing all numeric data types
df_new['TOTAL_AREA'] = df_new['TOTAL_AREA'].str.replace('[,$]','', regex=True).astype('float32')
df_new['BASE_AREA'] = df_new['BASE_AREA'].str.replace('[,$]','', regex=True).astype('float32')
df_new['BASEMENT_AREA'] = df_new['BASEMENT_AREA'].str.replace('[,$]','', regex=True).astype('float32')
df_new['BASEMENT_FINISHED_AREA'] = df_new['BASEMENT_FINISHED_AREA'].str.replace('[,$]','', regex=True).astype('float32')
df_new['PARKING_AREA'] = df_new['PARKING_AREA'].str.replace('[,$]','', regex=True).astype('float32')
df_new['LAND_VALUE'] = df_new['LAND_VALUE'].str.replace('[,$]','', regex=True).astype('float64')
df_new['LAST_SALE_PRICE'] = df_new['LAST_SALE_PRICE'].str.replace('[,$]','', regex=True).astype('float64')
df_new = df_new[df_new['LAST_SALE_PRICE'] != 0].copy().reset_index(drop=True)
df_new['MARKET_VALUE'] = df_new['MARKET_VALUE'].str.replace('[,$]','', regex=True).astype('float64')

# Extracting features from date values
df_new['LAST_SALE_DATE'] = pd.to_datetime(df_new['LAST_SALE_DATE'])

# Generating useful information from date values

df_new['SALE_DATE_YR'] = df_new['LAST_SALE_DATE'].dt.year
df_new['AGE_OF_HOUSE_AT_SALE'] = df_new['SALE_DATE_YR'] - df_new['YEAR_BUILT']
df_new['SALE_DATE_MNTH'] = df_new['LAST_SALE_DATE'].dt.month
df_new['SALE_DATE_DT'] = df_new['LAST_SALE_DATE'].dt.day


C:\Users\ssoma\Miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\ssoma\Miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [6]:
# Removed Items - 'QUALITY_CODE', 'ARCHITECTURAL_STYLE', 'SALE_DATE_MNTH', 'LOTS_SIZE', 'SALE_DATE_YR', 'BASEMENT_AREA',

df_new = df_new[[ 'BEDROOMS', 'BATHROOMS','HALF_BATHROOMS', 'TOTAL_ROOMS', 'TOTAL_AREA',
          'BASE_AREA',  'BASEMENT_FINISHED_AREA', 'PARKING_AREA','LAND_VALUE', 'LAST_SALE_PRICE','MARKET_VALUE',
           'AGE_OF_HOUSE_AT_SALE']].copy().drop_duplicates().reset_index(drop=True)

df_new = pd.get_dummies(df_new)
df_new.head()

,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,TOTAL_AREA,BASE_AREA,BASEMENT_FINISHED_AREA,PARKING_AREA,LAND_VALUE,LAST_SALE_PRICE,MARKET_VALUE,AGE_OF_HOUSE_AT_SALE
0,4,3,2,10,3304.0,2132.0,580.0,650.0,0.0,465606.0,480260.0,1
1,4,3,1,10,4000.0,2186.0,0.0,620.0,0.0,515000.0,521715.0,15
2,4,3,1,10,3952.0,2168.0,0.0,940.0,0.0,645000.0,528630.0,14
3,4,4,1,9,4252.0,2336.0,1073.0,620.0,0.0,499900.0,585120.0,12
4,3,2,0,9,2746.0,2746.0,0.0,620.0,0.0,375000.0,447057.0,12


# Data Transformation

In [7]:
# df_new['LAST_SALE_DATE'] = pd.to_datetime(df_new['LAST_SALE_DATE'], format="%b %d %Y %I:%M:%S:%f%p")

# # Generating useful information from date values

# df_new['SALE_DATE_YR'] = df_new['LAST_SALE_DATE'].dt.year
# df_new['AGE_OF_HOUSE_AT_SALE'] = df_new['SALE_DATE_YR'] - df_new['YRBLT']
# df_new['SALE_DATE_MNTH'] = df_new['LAST_SALE_DATE'].dt.month
# df_new['SALE_DATE_DT'] = df_new['LAST_SALE_DATE'].dt.day
# df_new.drop(['LAST_SALE_DATE', 'YRBLT', 'USPF', 'PHYS'], axis=1, inplace=True)
# df_new = pd.get_dummies(df_new)
# df_new.head()

In [8]:
from sklearn.model_selection import train_test_split

df_y = df_new['LAST_SALE_PRICE']
df_x = df_new.drop('LAST_SALE_PRICE', axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.25, random_state = 2)
print(f' Training dataset shape: {x_train.shape} | Testing dataset shape: {x_test.shape}')

 Training dataset shape: (126, 11) | Testing dataset shape: (43, 11)


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

model = LinearRegression()
model.fit(x_train, y_train)

# print(model.intercept_)
# print(model.coef_)
y_pred = model.predict(x_test)

df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(f'{df.head()} \n')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R-squared:', metrics.r2_score(y_test, y_pred, multioutput='uniform_average'))

# https://www.analyticsvidhya.com/blog/2021/05/all-you-need-to-know-about-your-first-machine-learning-model-linear-regression/
# https://www.analyticsvidhya.com/blog/2021/07/an-introduction-to-linear-regression/
# https://www.analyticsvidhya.com/blog/2021/06/getting-started-with-machine-learning%e2%80%8a-%e2%80%8aimplementing-linear-regression-from-scratch/


      Actual      Predicted
42  296617.0  281801.125920
23  553050.0  471531.194541
45  413000.0  330869.888205
48  282589.0  273153.494169
13  691795.0  463185.260712 

Mean Absolute Error: 41958.77922525622
Mean Squared Error: 3629211835.385946
Root Mean Squared Error: 60242.94012899724
R-squared: 0.7278110725062761


# Predicting unknown dataset

In [17]:
# Fixing the lot size
unseen_data['start'] = np.where(unseen_data.LOT_SIZE.str.find('AC') == -1, unseen_data.LOT_SIZE.str.split('X') , unseen_data.LOT_SIZE.str.replace(pat=" AC", repl=""))
unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1] = unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1].apply(lambda x: list(map(float, x)))
unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1]  = unseen_data['start'][unseen_data.LOT_SIZE.str.find('AC') == -1].apply(lambda x: np.prod(x[0:2])/43560)
unseen_data['LOTS_SIZE'] = pd.to_numeric(unseen_data['start'])

# Fixing all numeric data types
unseen_data['TOTAL_AREA'] = unseen_data['TOTAL_AREA'].str.replace('[,$]','', regex=True).astype('float32')
unseen_data['BASE_AREA'] = unseen_data['BASE_AREA'].str.replace('[,$]','', regex=True).astype('float32')
unseen_data['BASEMENT_AREA'] = unseen_data['BASEMENT_AREA'].str.replace('[,$]','', regex=True).astype('float32')
#unseen_data['BASEMENT_FINISHED_AREA'] = unseen_data['BASEMENT_FINISHED_AREA'].str.replace('[,$]','', regex=True).astype('float32')
# unseen_data['PARKING_AREA'] = unseen_data['PARKING_AREA'].str.replace('[,$]','', regex=True).astype('float32')
unseen_data['LAND_VALUE'] = unseen_data['LAND_VALUE'].str.replace('[,$]','', regex=True).astype('float64')
unseen_data['LAST_SALE_PRICE'] = unseen_data['LAST_SALE_PRICE'].str.replace('[,$]','', regex=True).astype('float64')
unseen_data = unseen_data[unseen_data['LAST_SALE_PRICE'] != 0].copy().reset_index(drop=True)
unseen_data['MARKET_VALUE'] = unseen_data['MARKET_VALUE'].str.replace('[,$]','', regex=True).astype('float64')

# Extracting features from date values
unseen_data['LAST_SALE_DATE'] = pd.to_datetime(unseen_data['LAST_SALE_DATE'])

# Generating useful information from date values

unseen_data['SALE_DATE_YR'] = dt.date.today().year
unseen_data['AGE_OF_HOUSE_AT_SALE'] = (dt.date.today().year)  - (unseen_data['YEAR_BUILT'])
unseen_data['SALE_DATE_MNTH'] = dt.date.today().month
unseen_data['SALE_DATE_DT'] = dt.date.today().day


unseen_data = unseen_data[[ 'BEDROOMS', 'BATHROOMS','HALF_BATHROOMS', 'TOTAL_ROOMS', 'TOTAL_AREA',
          'BASE_AREA',  'BASEMENT_FINISHED_AREA', 'PARKING_AREA','LAND_VALUE', 'MARKET_VALUE',
           'AGE_OF_HOUSE_AT_SALE']].copy().drop_duplicates().reset_index(drop=True)

unseen_data = pd.get_dummies(unseen_data)

C:\Users\ssoma\Miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ssoma\Miniconda3\envs\myDefaultEnv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
unseen_data = unseen_data_backup.copy()
unseen_data

,ACCOUNT,SITUS,CITY,ZIP,OWNER,MAILING_ADDR,MAILING_CITY,MAILING_STATE,MAILING_ZIP,SUBDIVISION,SCHOOL_DISTRICT,NEIGHBORHOOD_CODE,LOT_SIZE,YEAR_BUILT,QUALITY_CODE,BEDROOMS,BATHROOMS,HALF_BATHROOMS,TOTAL_ROOMS,ARCHITECTURAL_STYLE,TOTAL_AREA,BASE_AREA,PARKING_AREA,BASEMENT_AREA,BASEMENT_FINISHED_AREA,MARKET_VALUE,LAND_VALUE,LAST_SALE_PRICE,LAST_ADJUSTED_SALE_PRICE,LAST_SALE_DATE,LAST_SALE_QUALIFIED,LAST_SALE_UPSF
0,T070800695,1945 BRIDGECREST XING,St. Charles,63303,BAGOT PAUL*BAGOT JENA K,1945 BRIDGECREST XING,ST CHARLES,MO,63303,TALBRIDGE VLGS A-C,Francis Howell,1367,0.1400 AC,2010,R - 35 - Average to Good,4,2,1,7,2STY - 2 Story Building,"2,544","1,312",400,"1,232",0,"$318,216.00","$58,000.00","$315,000.00","$315,000.00",8/9/2019,Y,$123.82


In [10]:
unseen_data_backup = unseen_data.copy()

In [18]:
predicted_price = model.predict(unseen_data)
df = pd.DataFrame({'Address': address, 'Predicted': predicted_price})
df

,Address,Predicted
0,1945 BRIDGECREST XING,307196.979897


# Visualizations

In [ ]:
numericCols = df_new.select_dtypes(include=['float64','int64'])

# Calculate Correlation
corr = numericCols.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})